## goal

ChrInt_boxplot

In [95]:
import numpy as np
import pandas as pd
import hicstraw
from matplotlib.colors import LinearSegmentedColormap
from matplotlib import pyplot as plt
from matplotlib import gridspec
from matplotlib import backends
from matplotlib.backends.backend_pdf import PdfPages
import matplotlib.pyplot as plt 
import seaborn as sns
import math
from matplotlib import rcParams
import os
import re
import scipy

from statannot import add_stat_annotation
from statannotations.Annotator import Annotator

## take a look at  hic data

In [96]:
# hic = hicstraw.HiCFile("https://www.encodeproject.org/files/ENCFF718AWL/@@download/ENCFF718AWL.hic")
hic_file = "/home/qchen/PhD/Neu/OCEANC/Neu_H/hic_results/H_hicpro2juicebox/HL60_Ctrl.allValidPairs.hic"
hic_test = hicstraw.HiCFile(hic_file)



In [97]:
print(hic_test.getGenomeID())
print(hic_test.getResolutions())


/HiC-Pro_3.0.0/annotation/chrom_hg19.sizes
[2500000, 1000000, 500000, 250000, 100000, 50000, 25000, 10000, 5000]


#### 

In [98]:
chrom_length = {}
for chrom in hic_test.getChromosomes():
    # print(chrom.name, chrom.length)
    chrom_length[chrom.name] = chrom.length
chrom_length 


{'ALL': 3095693,
 '1': 249250621,
 '2': 243199373,
 '3': 198022430,
 '4': 191154276,
 '5': 180915260,
 '6': 171115067,
 '7': 159138663,
 '8': 146364022,
 '9': 141213431,
 '10': 135534747,
 '11': 135006516,
 '12': 133851895,
 '13': 115169878,
 '14': 107349540,
 '15': 102531392,
 '16': 90354753,
 '17': 81195210,
 '18': 78077248,
 '19': 59128983,
 '20': 63025520,
 '21': 48129895,
 '22': 51304566,
 'X': 155270560,
 'Y': 59373566,
 'M': 16571}

In [ ]:

del chrom_length["ALL"]

del chrom_length["M"]


In [102]:
chrom_length.keys()

dict_keys(['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', 'X', 'Y'])

In [ ]:

# extract All ChrInt of hub anchor associated with a gene (several genes)


def All_ChrInt_in_a_hub(hubs_anchor, hic_getMatrixZoomData_dict, chrom_length, norm, resolution):   

    contact = {}

    ## step 1: go through every anchors and locate the bin 

    for i in range(hubs_anchor.shape[0]):
        # two anchor in the same chr
        Chr = hubs_anchor.iloc[i,0]
        
        Start1 =  hubs_anchor.iloc[i,1]
        End1 = hubs_anchor.iloc[i,2]
        
        Start2 =  hubs_anchor.iloc[i,4]
        End2 = hubs_anchor.iloc[i,5]
    ## step 2: get the small matrix of the anchor
        
        # matrix_chr = hic.getMatrixZoomData( Chr[3:], Chr[3:], "oe", norm, "BP", resolution)
        matrix_chr = hic_getMatrixZoomData_dict[Chr]
        
        Start1_bin = int(Start1/resolution)*resolution
        End1_bin = int(End1/resolution)*resolution
        
        Start2_bin = int(Start2/resolution)*resolution
        End2_bin = int(End2/resolution)*resolution
        
        numpy_matrix_anchor = matrix_chr.getRecordsAsMatrix(Start1_bin, End1_bin ,Start2_bin, End2_bin)
        # numpy_matrix_anchor : does not necessarily symmetric. only if two anchors are the same length
        
        print(numpy_matrix_anchor.shape)
        

    ## step 3: turn the symmetric matrix to arrary & save the data to the dict
        Gene = hubs_anchor.iloc[i,6]
   
        if numpy_matrix_anchor.shape[0] == numpy_matrix_anchor.shape[1]:
            contact[Gene] = numpy_matrix_anchor[np.triu_indices(numpy_matrix_anchor.shape[0])]
            print(Gene + " has " + str(len(numpy_matrix_anchor[np.triu_indices(numpy_matrix_anchor.shape[0])])))
        ##---- rectangle ----
        else: 
            contact[Gene] = numpy_matrix_anchor.flatten()
            print(Gene + " has " + str(len(numpy_matrix_anchor.flatten())))
        

    print("Number of genes we are ploting: "+str(len(contact)))


    return  contact

## run on the data

In [116]:
! ls /home/qchen/PhD/Neu/OCEANC/Neu_H/hic_results/H_hicpro2juicebox | grep -v "rep" | grep ".allValidPairs.hic"

HL60_2d.allValidPairs.hic
HL60_4d.allValidPairs.hic
HL60_4h.allValidPairs.hic
HL60_Ctrl.allValidPairs.hic


In [117]:
# here DO NOT add \ in My Drive

Gene_asso_hub_file = '/home/qchen/PhD/Neu/Gene_asso_region_ChrInt_boxplot/Gene_asso_region_ForPlot_add.csv'
# Gene_asso_hub_file = '/home/qchen/PhD/ExhaustedT/hichub_asso_CTCF_ATAC/'

hic_path = '/home/qchen/PhD/Neu/OCEANC/Neu_H/hic_results/H_hicpro2juicebox/'
# ! ls /home/qchen/PhD/Neu/OCEANC/Neu_H/hic_results/H_hicpro2juicebox | grep -v "rep" | grep ".allValidPairs.hic"

In [118]:
! ls $Gene_asso_hub_file

/home/qchen/PhD/Neu/Gene_asso_region_ChrInt_boxplot/Gene_asso_region_ForPlot_rm_4d.csv


In [ ]:

norm = "KR"
resolution = 10**4


In [ ]:

hubs_anchor = pd.read_csv(Gene_asso_hub_file,header = 0)

## calculate all ChrInt in 4 conditions

In [124]:
# ls /home/qchen/PhD/ExhaustedT/Hic/results/hic_hicpro2juicebox |grep -v "dist"|grep "m.hic"

! ls /home/qchen/PhD/Neu/OCEANC/Neu_H/hic_results/H_hicpro2juicebox | grep -v "rep" | grep ".allValidPairs.hic"

HL60_2d.allValidPairs.hic
HL60_4d.allValidPairs.hic
HL60_4h.allValidPairs.hic
HL60_Ctrl.allValidPairs.hic


In [125]:
Hic_files = [f for f in os.listdir(hic_path) if re.match(r'.*.allValidPairs.hic', f)]
Hic_files

['HL60_4d.allValidPairs.hic',
 'HL60_4h.allValidPairs.hic',
 'HL60_Ctrl_rep2.allValidPairs.hic',
 'HL60_2d_rep1.allValidPairs.hic',
 'HL60_4h_rep1.allValidPairs.hic',
 'HL60_4h_rep2.allValidPairs.hic',
 'HL60_2d.allValidPairs.hic',
 'HL60_4d_rep1.allValidPairs.hic',
 'HL60_Ctrl_rep1.allValidPairs.hic',
 'HL60_2d_rep2.allValidPairs.hic',
 'HL60_4d_rep2.allValidPairs.hic',
 'HL60_Ctrl.allValidPairs.hic']

In [128]:
# myorder = [3,0,2,1,4]
myorder = [11,1,6,0]
# myorder = [11,1,6]
Hic_files = [Hic_files[i] for i in myorder]
Hic_files

['HL60_Ctrl.allValidPairs.hic',
 'HL60_4h.allValidPairs.hic',
 'HL60_2d.allValidPairs.hic',
 'HL60_4d.allValidPairs.hic']

In [133]:
for i in range(len(Hic_files)):
    print(i)
    print(Hic_files[i])

0
HL60_Ctrl.allValidPairs.hic
1
HL60_4h.allValidPairs.hic
2
HL60_2d.allValidPairs.hic
3
HL60_4d.allValidPairs.hic


In [ ]:


Hic_contact_list = []

    
for H in Hic_files:
    print(H)
    # 1. for each hic file, calculated the gene asso hub ChrInt 
    print(H.split('.')[0])
    
    #  ---> load corresponding hic data
    # hic = hicstraw.HiCFile(hic_path + H )
    hic_getMatrixZoomData_dict = hic_getMatrixZoomData(hic_path + H)

    
    Hic_contact_list.append(All_ChrInt_in_a_hub(hubs_anchor, hic_getMatrixZoomData_dict, chrom_length, norm, resolution))
    

HL60_Ctrl.allValidPairs.hic
HL60_Ctrl
chr1
chr2
chr3
chr4
chr5
chr6
chr7
chr8
chr9
chr10
chr11
chr12
chr13
chr14
chr15
chr16
chr17
chr18
chr19
chr20
chr21
chr22
chrX
chrY
(10, 10)
CYBB has 55
(24, 24)
CCL2_1 has 300
(8, 10)
CCL2_2 has 80
(14, 7)
CCL2_3 has 98
(8, 8)
CCL2_4 has 36
(10, 10)
CEBPE_1 has 55
(22, 22)
CEBPE_2 has 253
(22, 22)
ALOX5AP has 253
(50, 50)
CEBPB_1 has 1275
(28, 28)
CEBPB_2 has 406
(7, 36)
CEBPB_3 has 252
(10, 10)
IRF5 has 55
(9, 9)
CEBPA has 45
Number of genes we are ploting: 13
HL60_4h.allValidPairs.hic
HL60_4h
chr1
chr2
chr3
chr4
chr5
chr6
chr7
chr8
chr9
chr10
chr11
chr12
chr13
chr14
chr15
chr16
chr17
chr18
chr19
chr20
chr21
chr22
chrX
chrY
(10, 10)
CYBB has 55
(24, 24)
CCL2_1 has 300
(8, 10)
CCL2_2 has 80
(14, 7)
CCL2_3 has 98
(8, 8)
CCL2_4 has 36
(10, 10)
CEBPE_1 has 55
(22, 22)
CEBPE_2 has 253
(22, 22)
ALOX5AP has 253
(50, 50)
CEBPB_1 has 1275
(28, 28)
CEBPB_2 has 406
(7, 36)
CEBPB_3 has 252
(10, 10)
IRF5 has 55
(9, 9)
CEBPA has 45
Number of genes we are plot

In [135]:
len(Hic_contact_list)

4


# plot


In [ ]:

## save all the figures in a loop to one pdf file. 
from matplotlib.backends.backend_pdf import PdfPages
pdf = PdfPages("./Gene_asso_region_ChrInt.pdf")


for Gene in  hubs_anchor["Gene"]:
    print(Gene)
    
    df = pd.DataFrame({"0h" : Hic_contact_list[0][Gene],
                     "4h" : Hic_contact_list[1][Gene],
                     "2d" : Hic_contact_list[2][Gene],
                      "4d" : Hic_contact_list[3][Gene]})
    
    df.to_csv("./" + Gene + "region_ChrInt.csv") 
    
    df2 = pd.melt(df,var_name='Condition', value_name='ChrInt')
    
    # draw boxplot

    ## chnage boxplot color:
    PROPS = {
        'boxprops':{'facecolor':'none', 'edgecolor':'black'},
        'medianprops':{'color':'black'},
        'whiskerprops':{'color':'black', 'linestyle':'--'},
        'capprops':{'color':'black'}
    }
    # 1. horizontal ---- but hard to add p value 
    plt.figure(figsize=(6,4.5))
    ax1 = sns.boxplot(x='ChrInt',y='Condition', data=df2, color = "white",width = 0.4,showfliers=False,**PROPS)
    sns.stripplot(x='ChrInt',y='Condition', data=df2, color = "none", dodge=True,size = 2,jitter = 0.15,ax = ax1 )

    ax1.set_title(Gene)


    # ---------Remove the top and right spines from plot(s)---------
    sns.despine(fig=None, ax=None, top=True, right=True, left=False, bottom=False, offset=None, trim=False)

    # plt.show()
    

    pvalue1 = scipy.stats.ranksums(df2[df2.Condition == "0h"]["ChrInt"].dropna(),df2[df2.Condition == "4h"]["ChrInt"].dropna())
    print( "0h vs 4h "  + str( pvalue1))

    pvalue2 = scipy.stats.ranksums(df2[df2.Condition == "0h"]["ChrInt"].dropna(),df2[df2.Condition == "2d"]["ChrInt"].dropna())
    print( "0h vs 2d  "  + str( pvalue2))
    
    pvalue3 = scipy.stats.ranksums(df2[df2.Condition == "0h"]["ChrInt"].dropna(),df2[df2.Condition == "4d"]["ChrInt"].dropna())
    print( "0h vs 4d  "  + str( pvalue2))
    
    pdf.savefig() 

    
    # 2. Vertical ----easy to add p value
    plt.figure(figsize=(4.5,6))
    
    ax2 = sns.boxplot(x='Condition', y='ChrInt', data=df2, color = "white",width = 0.4,showfliers=False,**PROPS)
    sns.stripplot(x='Condition', y='ChrInt', data=df2, color = "none", dodge=True,size = 2,jitter = 0.15,ax = ax2)
    
    
    ax2.set_title(Gene)
    # MOVE Y AXIS TO THE RIGHT 
    ax2.yaxis.set_label_position("right")
    ax2.yaxis.tick_right()
    # ---------Remove the top andLEFT spines from plot(s)---------
    sns.despine(fig=None, ax=None, top=True, right=False, left= True, bottom=False, offset=None, trim=False)

    from statannot import add_stat_annotation


    add_stat_annotation(ax2, data=df2,x='ChrInt',y='Condition',
                        box_pairs=[("0h", "4h"), ("0h", "2d"),("0h", "4d")],
                        test='Wilcoxon', text_format='full', loc='inside')
    
    pdf.savefig() 
    
    # 3. anova test
    plt.figure(figsize=(4.5,6))
    
    ax2 = sns.boxplot(x='Condition', y='ChrInt', data=df2, color = "white",width = 0.4,showfliers=False,**PROPS)
    sns.stripplot(x='Condition', y='ChrInt', data=df2, color = "none", dodge=True,size = 2,jitter = 0.15,ax = ax2)
    
    # ax2.set_title(Gene)
    # MOVE Y AXIS TO THE RIGHT 
    ax2.yaxis.set_label_position("right")
    ax2.yaxis.tick_right()
    # ---------Remove the top andLEFT spines from plot(s)---------
    sns.despine(fig=None, ax=None, top=True, right=False, left= True, bottom=False, offset=None, trim=False)

    # stats f_oneway functions takes the groups as input and returns ANOVA F and p value
    import scipy.stats as stats
    fvalue, pvalue = stats.f_oneway(df['0h'], df['4h'], df['2d'], df['4d'])
    print(fvalue, pvalue)
    significance = "*" if pvalue < 0.05 else "ns"
    
    T= Gene+" \n"+ f'ANOVA p-value: {pvalue:.4f} {significance}'
    ax2.set_title(T)

    pdf.savefig() 

    
pdf.close()

CYBB
0h vs 4h RanksumsResult(statistic=-0.5529784118452927, pvalue=0.5802781928282779)
0h vs 2d  RanksumsResult(statistic=-1.6828478155075666, pvalue=0.09240455591213687)
0h vs 4d  RanksumsResult(statistic=-1.6828478155075666, pvalue=0.09240455591213687)
Using zero_method  wilcox
0h v.s. 4h: Wilcoxon test (paired samples) with Bonferroni correction, P_val=5.996e-01 stat=6.170e+02
Using zero_method  wilcox
0h v.s. 2d: Wilcoxon test (paired samples) with Bonferroni correction, P_val=8.112e-03 stat=4.120e+02
Using zero_method  wilcox
0h v.s. 4d: Wilcoxon test (paired samples) with Bonferroni correction, P_val=1.000e+00 stat=6.780e+02
1.1172130534195424 0.34299679463060395
CCL2_1
0h vs 4h RanksumsResult(statistic=-1.6631439865907918, pvalue=0.09628361679032632)
0h vs 2d  RanksumsResult(statistic=-1.8468387344725274, pvalue=0.06477051618298572)
0h vs 4d  RanksumsResult(statistic=-1.8468387344725274, pvalue=0.06477051618298572)
Using zero_method  wilcox
0h v.s. 4h: Wilcoxon test (paired samp

/tmp/ipykernel_26765/3605437427.py:81: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  plt.figure(figsize=(4.5,6))


ALOX5AP
0h vs 4h RanksumsResult(statistic=-1.06993006993007, pvalue=0.28465078680354083)
0h vs 2d  RanksumsResult(statistic=-0.567649741562785, pvalue=0.5702728243961538)
0h vs 4d  RanksumsResult(statistic=-0.567649741562785, pvalue=0.5702728243961538)
Using zero_method  wilcox
0h v.s. 4h: Wilcoxon test (paired samples) with Bonferroni correction, P_val=3.977e-01 stat=1.431e+04
Using zero_method  wilcox
0h v.s. 2d: Wilcoxon test (paired samples) with Bonferroni correction, P_val=1.000e+00 stat=1.536e+04
Using zero_method  wilcox
0h v.s. 4d: Wilcoxon test (paired samples) with Bonferroni correction, P_val=9.403e-01 stat=1.489e+04
0.4669970548445846 0.7053677133801486
CEBPB_1
0h vs 4h RanksumsResult(statistic=0.6085863259104622, pvalue=0.5427986703665875)
0h vs 2d  RanksumsResult(statistic=-2.8199061372444687, pvalue=0.004803769860372517)
0h vs 4d  RanksumsResult(statistic=-2.8199061372444687, pvalue=0.004803769860372517)
Using zero_method  wilcox
0h v.s. 4h: Wilcoxon test (paired sample